# Firts attemp to charge the db and transfer learning 

The dataset contains a number of different subsets of the full food-101 data. The idea is to make a more exciting simple training set for image analysis than CIFAR10 or MNIST. For this reason the data includes massively downscaled versions of the images to enable quick tests. The data has been reformatted as HDF5 and specifically Keras HDF5Matrix which allows them to be easily read in. The file names indicate the contents of the file. For example

foodc101n1000_r384x384x3.h5 means there are 101 categories represented, with n=1000 images, that have a resolution of 384x384x3 (RGB, uint8)

foodtestc101n1000r32x32x1.h5 means the data is part of the validation set, has 101 categories represented, with n=1000 images, that have a resolution of 32x32x1 (float32 from -1 to 1)

## VGG16

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h5py
import skimage
import matplotlib.pyplot as plt
import sys


In [0]:
%cd /content/drive/My Drive/deeplearning/2020.deeplearning/U3project/food

/content/drive/My Drive/deeplearning/2020.deeplearning/U3project/food


In [0]:
f=h5py.File('food_c101_n10099_r64x64x3.h5','r')
print(list(f.keys()))
print(len(f["category"]))
print(len(f["category_names"]))
print(len(f["images"]))

In [0]:
len(f["images"][0][0])

In [0]:
print([int(i) for i in f["category"][0]])
print(f["images"][0].shape)
fig=plt.figure(figsize=(20,20))
n=25
col=5
for i in range(n):
    ax=fig.add_subplot(n/col,col,i+1)
    #ax.set_title(f["category_names"][i].decode())
    ax.imshow(f["images"][i])
plt.savefig("./sample_show_64x64")

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras import optimizers

In [0]:
#model = ResNet50(weights=None,input_shape=(384,384,3),classes=101)
model = VGG16(weights=None,input_shape=(64,64,3),classes=101)


In [0]:
model.summary()

In [0]:
x=np.array(f["images"])/255.
y=np.array([[int(i) for i in f["category"][j]] for j in range(len(f["category"]))])
#model.compile(loss='categorical_crossentropy',optimizer=optimizers.rmsprop(lr=0.0001, decay=1e-6))


In [0]:
x.shape

(10099, 64, 64, 3)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=0.00008, beta_1=0.9, beta_2=0.97, epsilon=1e-7))
from sklearn.model_selection import train_test_split
train_x,test_x, train_y, test_y = train_test_split(x,y,test_size = 0.2)

In [0]:
model.fit(train_x[:128],train_y[:128],batch_size=128,epochs=90,shuffle=False)


Epoch 1/90
128/128 [==============================] - 22s 172ms/step - loss: 4.3253
Epoch 2/90
128/128 [==============================] - 22s 170ms/step - loss: 4.2966
Epoch 3/90
128/128 [==============================] - 22s 171ms/step - loss: 4.2675
Epoch 4/90
128/128 [==============================] - 22s 171ms/step - loss: 4.2512
Epoch 5/90
128/128 [==============================] - 22s 170ms/step - loss: 4.2513
Epoch 6/90
128/128 [==============================] - 22s 170ms/step - loss: 4.2468
Epoch 7/90
128/128 [==============================] - 22s 170ms/step - loss: 4.2233
Epoch 8/90
128/128 [==============================] - 22s 170ms/step - loss: 4.1954
Epoch 9/90
128/128 [==============================] - 22s 171ms/step - loss: 4.1788
Epoch 10/90
128/128 [==============================] - 22s 171ms/step - loss: 4.1754
Epoch 11/90
128/128 [==============================] - 22s 171ms/step - loss: 4.1786
Epoch 12/90
128/128 [==============================] - 22s 171ms/step - lo

In [0]:
test_x=train_x[:50]
test_y=train_y[:50]
pred_y=model.predict(test_x)
zero_y=np.zeros(pred_y.shape)
argmax_lst=np.argmax(pred_y,axis=1)

In [0]:
for i in range(len(argmax_lst)):
    zero_y[i][argmax_lst[i]]=1
pred_y=zero_y
from sklearn.metrics import f1_score,accuracy_score

In [0]:
print("Acc-Score:",accuracy_score(np.array(test_y),np.array(pred_y)))


Acc-Score: 0.04


In [0]:
argmax_lst

array([51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51])

In [0]:
np.argmax(test_y,axis=1)

array([ 85,  58,   0,  46,  70,  29,  70,  74,  35,  59,  40,  97,  31,
        98,  65,  28,  51,  47,  62,  17,   9,  81,  56,  52,  51,  13,
        75,   4,  36,  18,  12,  44,  69,   6,  81, 100,  18,  34,   7,
        58,  88,  63,  84,  61,  20,  98,  11,  13,  41,  44])

## Inception

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras import optimizers
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


In [0]:
%cd /content/drive/My Drive/deeplearning/2020.deeplearning/U3project/food

/content/drive/My Drive/deeplearning/2020.deeplearning/U3project/food


In [0]:
train_data_dir='prueba'

In [0]:
TEST_SPLIT = 0.25
epochs = 80
#batch_size = 16
batch_size=32
MICRO_DATA = True # very small subset (just 3 groups)
SAMPLE_TRAINING = False # make train set smaller for faster iteration
IMG_SIZE = (112, 112)
LEARNING_RATE = 3e-2
seed=1

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255.,
    horizontal_flip=False,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', seed=seed) # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', seed=seed) # set as validation data

Found 8037 images belonging to 10 classes.
Found 2009 images belonging to 10 classes.


In [0]:
from keras import applications
model = applications.InceptionV3(weights='imagenet', 
                                include_top=False, input_shape = (112,112,3))


87916544/87910968 [==============================] - 6s 0us/step


In [0]:
for layer in model.layers[:17]:
    layer.trainable = False
regparameter=0.0001

x = model.output
x = Flatten()(x)
x = Dense(256, activation="relu",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=regparameter,l2=regparameter))(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation="relu")(x)
predictions = Dense(8, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
model_final.summary()

In [0]:
model_final=tf.keras.models.load_model('Xception_finetuned_tenclasses')

In [0]:
model_final=tf.keras.models.load_model('vgg_finetuned_tenclasses.model')

In [0]:
chkpt = tf.keras.callbacks.ModelCheckpoint(filepath='./best_model_todate', save_best_only=True, save_weights_only=True)

In [0]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 1,callbacks=[chkpt])

Instructions for updating:
Please use Model.fit, which supports generators.
251/251 [==============================] - 5586s 22s/step - loss: 1.1951 - accuracy: 0.7253 - val_loss: 1.6983 - val_accuracy: 0.6084


In [0]:
p_pr=model_final.predict_generator(generator=validation_generator)


In [0]:
y_pred=np.argmax(p_pr,axis=1)
y_true = test_generator.classes